# Avaliação Continuada 2 (AC2) - Dados de Treinamento

Abaixo, iremos criar um modelo para classificar mensagens de spam a partir de um dataset obtido no Kaggle.

## Imports

Primeiro, começamos a importar todas as bibliotecas que iremos precisar nesse notebook, apenas para simplificar mais tarde.

In [1]:
# Leitura dos módulos
from __future__ import print_function

# biblioteca usada para trabalhar com dataframes
import pandas as pd
from pandas import Series,DataFrame

# uma biblioteca usada para auxiliar a limpeza de tags html
import html

# bibliotecas básicas como numpy e matplotlib para exibir 
# alguns gráficos caso seja necessário. 
import numpy as np
import matplotlib.pyplot as plt

# biblioteca para usar para medir acuracia e separar os dados de treino e teste
from sklearn import svm
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

# biblioteca usada para criar modelos de word embedding
import gensim
from gensim.test.utils import common_texts
from gensim.models import Word2Vec

np.random.seed(1000)

import re # biblioteca para expressoes regulares

import nltk # biblioteca para Processamento de Linguagem Natural
from nltk.stem.porter import PorterStemmer # para fazer a estemização em documentos da lingua inglesa
from nltk.stem import RSLPStemmer # para fazer a estemização em documentos da lingua portuguesa

Abaixo, um import que deve ser executado apenas um vez, caso não tenha executado, descomente a linha e execute novamente.

In [2]:
# nltk.download('all') # instala todos os recursos da biblioteca NLTK. Você pode descomentar esta linha na 1a vez que for executar este notebook

## Dataset

Depois, inicializamos o nosso dataset que foi baixado do Kaggle e pode ser obtido a partir [desse link](https://www.kaggle.com/team-ai/spam-text-message-classification).

In [3]:
data_set_url = "https://raw.githubusercontent.com/H4ad/h4ad.facens.artificial-inteligence.text-classification/master/datasets/spam-text-message-20170820.csv"
data_set = pd.read_csv(data_set_url,sep=",",header=0)

## Análise Exploratória

Abaixo, um pouco dos dados que iremos ver e tratar durante o notebook.

### O que há dentro?

Vamos começar primeiro a visualizar o que há dentro desse dataset com alguns comandos que nos mostram alguns informações básicas.

In [4]:
display(data_set.head(n=5))
display(data_set.describe())

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


### Valores Duplicados

Na tabela acima, podemos ver que na coluna "Message", temos um total de 5572 valores mas apenas 5157 são únicos.

Para não influenciar ou causar algum problema durante os treinos, rodar um outro comando para realmente ter a contagem de quantos dados duplicados nós temos:

In [5]:
data_set.duplicated().sum()

415

O número é de 415, é bem expressivo. Dessa forma, vamos agora tratar de apagar esses valores duplicados.

In [6]:
print('Shape antes de remover os dados.')
print(data_set.shape)

data_set_cleaned = data_set.drop_duplicates(inplace=False)

print('\nShape após remover os dados.')
print(data_set_cleaned.shape)

print('\nQuantidade de itens removidos:', (data_set.shape[0] - data_set_cleaned.shape[0]))

Shape antes de remover os dados.
(5572, 2)

Shape após remover os dados.
(5157, 2)

Quantidade de itens removidos: 415


### Valores Faltando

Após checar valores duplicados, podemos também verificar se não há algum valor faltando, usando o comando abaixo:

In [7]:
data_set_cleaned.isna().sum() / data_set_cleaned.shape[0]

Category    0.0
Message     0.0
dtype: float64

E como podemos ver, tudo zero para ambas as colunas, dessa forma, não precisamos fazer mais nada já que não há valores nulos ou faltando.

## Pré-Processamento

Após essa analise inicial dos dados, vamos começar com a etapa de pré-processamento dos textos obtidos.

As técnicas e o código abaixo usado foi obtido e modificado a partir do código original escrito pelo professor Renato Moraes Silva que pode ser encontrado [nesse notebook.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232)


Sobre as técnicas, serão usadas as seguintes:

 - deixar todas as palavras com letras minúsculas
 - substituir os números pela tag *\<NUMBER\>*
 - substituir todas as URLS pela tag *\<URL\>*
 - substituir todos os emails pela tag *\<EMAIL\>*
 - substituir o símbolo de moeda pela tag *\<MONEY\>*
 - substituir todos os caracteres não-alfanuméricos por um espaço em branco

E o que foi adicionado além das técnicas acima foram:
 - remover as entidades do html (`&gt;` `&#62;`).

Além disso, será aplicado também um processo de *estemização*, no qual diminui uma palavra para o seu radical, tornando uma palavra como "flies" para "fli". 

E também, será aplicado um processo de remoção das *stopwords*, no qual remove palavras muito comuns de uma lingua. Como estamos usando um dataset em inglês, palavras como "i", "me", "my", "myself", "we" e muitas outras.

> Uma lista de exemplos mais detalhadas pode ser encontrado [nesse gist](https://gist.github.com/sebleier/554280).

In [8]:
def preprocessing(text, stemming = False, stopwords = False):
    """
    Funcao usada para fazer o tratamento de textos da lingua inglesa
    
    Parametros: 
        text: variavel do tipo string que contem o texto que devera ser tratado
        
        stemming: variavel do tipo booleana que indica se a estemizacao deve ser aplicada ou nao
        
        stopwords: variavel do tipo booleana que indica se as stopwords devem ser removidas ou nao
    """
    
    # Lower case
    text = text.lower()
    
    # remove tags HTML
    regex = re.compile('<[^<>]+>')
    text = re.sub(regex, " ", text) 
    
    # remove as entidades do HTML
    regex = re.compile('(&.+;)')
    text = re.sub(regex, " ", text) 
    
    # normaliza as URLs
    regex = re.compile('(http|https)://[^\s]*')
    text = re.sub(regex, "<URL>", text)

    # normaliza emails
    regex = re.compile('[^\s]+@[^\s]+')
    text = re.sub(regex, "<EMAIL>", text)
    
    #normaliza o símbolo de dólar
    regex = re.compile('[$]+')
    text = re.sub(regex, "<MONEY>", text)
    
    # converte todos os caracteres não-alfanuméricos em espaço
    regex = re.compile('[^A-Za-z0-9]+')  
    text = re.sub(regex, " ", text)
    
    # normaliza os numeros 
    regex = re.compile('[0-9]+')
    text = re.sub(regex, "<NUMBER>", text)
    
    # substitui varios espaçamentos seguidos em um só
    text = ' '.join(text.split())

    # remove stopwords
    if stopwords:
        words = text.split() # separa o texto em palavras
        words = [w for w in words if not w in nltk.corpus.stopwords.words('english')]
        text = " ".join( words )
    
    # aplica estemização
    if stemming: 
        stemmer_method = PorterStemmer()  
        words = text.split() # separa o texto em palavras
        words = [ stemmer_method.stem(w) for w in words ]
        text = " ".join( words )
    
    # remove palavras de apenas um caracter
    words = text.split() # separa o texto em palavras
    words = [ w for w in words if len(w)>1 ]
    text = " ".join( words )
    
    return text

Com a função acima, podemos agora chamar ela para realizar todo o pré-processamento do nosso dataset.

In [9]:
print("Um dos textos antes do processamento:")
display(data_set_cleaned["Message"][201])

data_set_processed = data_set_cleaned.copy()
data_set_processed["Message"] = data_set_processed["Message"].apply(preprocessing)

print("O mesmo texto, só que agora processado:")
display(data_set_processed["Message"][201])

Um dos textos antes do processamento:


'I sent you  &lt;#&gt;  bucks'

O mesmo texto, só que agora processado:


'sent you bucks'

## Word Embedding (Transformando texto em número)

Após realizar o pré-processamento, temos todos os nossos textos e palavras bem organizadas e formatadas. Dessa forma, o que devemos fazer é transformar essas palavras em um formato que o nosso modelo de decisão em arvore irá conseguir entender, e é ai que entra o Word Embedding.

Word Embedding é uma técnica para transformar um texto em um vetor de atributos com valores numericos. Há alguns pré-treinados, como o da Google, mas iremos criar um próprio porque os pré-treinados são gigantescos (na ordem de Gigas) e serão muito potentes para um simples experimento.

Assim, iremos começar agora a criar uma lista de lista com as palavras de cada mensagem no dataset.

In [21]:
sentences = data_set_processed["Message"].apply(lambda message: message.split()).tolist()

print("Algumas palavras que iremos usar para montar o nosso vocabulário:")
print(sentences[0])

Algumas palavras que iremos usar para montar o nosso vocabulário:
['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'great', 'world', 'la', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']


E com essa lista de palavras, iremos agora treinar o modelo de Word Embedding utilizando a biblioteca [Genshim](https://radimrehurek.com/gensim/models/word2vec.html).

In [11]:
model = Word2Vec(sentences=sentences, vector_size=200, window=3, min_count=2, workers=4)

Com o modelo treinado, podemos ver qual é o tamanho do vocabulário da seguinte forma:

In [12]:
print('Tamanho do Vocabulario: ', + len(model.wv))

Tamanho do Vocabulario:  3632


E se precisarmos obter o valor associado a cada palavra, podemos usar o seguinte código:

In [13]:
print(model.wv["call"])

[ 0.17709242 -0.12600343 -0.08791727  0.16236502  0.28659806 -0.35685495
  0.19971345  0.65286946 -0.19657624  0.24297376 -0.24326402 -0.39725733
  0.12053519  0.20995583 -0.08815522 -0.35241863 -0.08457894 -0.01442036
  0.02464742 -0.509421    0.22960162 -0.31073946 -0.1612563  -0.12143413
  0.09649967 -0.23418328 -0.04336007 -0.14399362 -0.34557962  0.15271045
  0.4199807   0.01149173  0.16687284 -0.08555082 -0.08175757  0.23091978
  0.29245043 -0.14143606 -0.07441217 -0.520712   -0.3377079  -0.09030245
  0.00683281  0.11942589  0.5297194  -0.14374237 -0.22596823 -0.09825926
  0.1655615   0.21038999  0.00480481 -0.15803435 -0.22627462 -0.30204716
  0.08603162 -0.0929756  -0.01578389 -0.31015152 -0.31742713  0.14342752
 -0.00655344  0.04587211 -0.00159489 -0.06823715 -0.62862366  0.15215392
  0.01981192  0.6696914  -0.31566703  0.5178813  -0.02604795  0.01275099
  0.36037377  0.03543808  0.11258061  0.20872577  0.29203978 -0.2529166
 -0.5108547  -0.05823629 -0.22444083 -0.07205907 -0.

E é possível até mesmo verificar a similaridade dessa palavra com outras, da seguinte forma:

In [14]:
print(model.wv.most_similar("call", topn=10))

[('claim', 0.9994510412216187), ('prize', 0.9992076754570007), ('free', 0.9992011785507202), ('cash', 0.9991630911827087), ('or', 0.9989598989486694), ('guaranteed', 0.998924195766449), ('<NUMBER>p', 0.9988560676574707), ('reply', 0.9988375902175903), ('txt', 0.9988107085227966), ('stop', 0.998788595199585)]


Dessa forma, agora que foi visto tudo que podemos fazer usando o modelo de Word Embedding, podemos usar ele para transformar todo o nosso dataset para um formato que será possível usar ele para o treinamento.

Vamos começar definindo uma função que irá transformar uma sentença inteira um vetor de números.

> A função abaixo é uma ligeira adaptação do código do professor Renato, e pode ser encontrado [nesse link.](https://facens.instructure.com/courses/7531/files/1356091?module_item_id=269232) 

In [15]:
def getDocvector(sentence):
  sentenceValues = []

  for word in sentence:
    # É interessante notar o uso do try aqui dentro,
    # ele se dá porque, durante a criação do modelo,
    # nós especificamos que ele só gere o modelo com
    # palavras que aparecem ao menos 2 vezes. Dessa 
    # forma, palavras com frequencia menor que isso
    # são simplesmente ignoradas, e ai podem causar 
    # que ao tentar acessar elas, ocorra uma exceção
    # porque ela não existe no modelo.
    try:
      sentenceValues.append(model.wv[word])
    except:
      pass

  if len(sentenceValues) > 0:
    return np.mean(sentenceValues, axis = 0)

  return np.zeros(model.wv.vector_size)

print('Testando com uma frase qualquer')
print(getDocvector(sentences[0]))

Testando com uma frase qualquer
[ 0.09511602 -0.07759578 -0.06158502  0.07376378  0.19277689 -0.20688233
  0.07275587  0.35907674 -0.122722    0.11012122 -0.15914634 -0.20448256
  0.07375097  0.1316086  -0.04187575 -0.1842224  -0.06262719 -0.00350946
  0.00449819 -0.27489054  0.1157137  -0.14582513 -0.06547634 -0.08518134
  0.05659532 -0.12771186 -0.03049851 -0.08392287 -0.18977188  0.10628672
  0.22717555  0.00103121  0.07097273 -0.0234408  -0.03558218  0.12889321
  0.13224864 -0.07967652 -0.03841361 -0.31163293 -0.1613926  -0.02067659
  0.00953143  0.06929825  0.3083307  -0.05120663 -0.122779   -0.08409563
  0.09526726  0.10617401  0.01536868 -0.10533574 -0.12229341 -0.15787761
  0.03991915 -0.03350319 -0.02107173 -0.16109027 -0.17343828  0.06413537
  0.00376832  0.01990785 -0.0126125  -0.03591307 -0.33706772  0.0938308
  0.01945116  0.3537798  -0.1846274   0.29767275 -0.02473962  0.01160061
  0.2064071   0.01261324  0.06667463  0.10853186  0.16218923 -0.16665506
 -0.315526   -0.0094

Com a função acima, podemos agora transformar todas as nossas mensagens do modelo para um vetores de valores, assim como, separar nossos valores de classificação e valores de treino e teste, da seguinte forma:

- x = Valores que podem ser usados para descrever se é spam ou não
- y = A informação se é realmente um spam

In [16]:
y = data_set_processed["Category"].to_numpy()
x = []

for sentence in data_set_processed["Message"]:
  x.append(getDocvector(sentence.split()))
  
x = np.array(x)

print("Shape do vetor de classe: ", y.shape)
print("Shape do vetor de valores: ", x.shape)

Shape do vetor de classe:  (5157,)
Shape do vetor de valores:  (5157, 200)


## Separando os Dados

Bom, após ter o modelo para poder transformar os textos em números, podemos agora começar a treinar o nosso modelo, mas antes, precisamos separar os dados que serão usados no treino.

Vamos separar os valores em:

- x_train = Valores de treino
- y_train = A classe se diz se é spam ou não de treino
- x_test = Valores para testar o modelo após o treino
- y_test = A classe dizendo se é spam ou não de teste para testar o modelo após o treino

In [17]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1000)

Além disso, vamos nos certificar que a tipagem dos valores estejam corretos, ou seja, todos sejam do tipo float.

In [18]:
x_train = x_train.astype(np.float64)
x_test = x_test.astype(np.float64)

Por fim, temos as seguintes métricas sobre os nossos dados de teste e treino.

In [19]:
print("Qntd. de amostras de treino: ", x_train.shape[0])
print("Qntd de amostras de teste: ", x_test.shape[0])

print("\nQtd. de dados de cada classe (treinamento)")
print("\tClasse ham: ", len(y_train[y_train == 'ham']))
print("\tClasse spam: ", len(y_train[y_train == 'spam']))

print("\nQtd. de dados de cada classe (teste)")
print("\tClasse ham: ", len(y_test[y_test == 'ham']))
print("\tClasse spam: ", len(y_test[y_test == 'spam']))

Qntd. de amostras de treino:  3609
Qntd de amostras de teste:  1548

Qtd. de dados de cada classe (treinamento)
	Classe ham:  3164
	Classe spam:  445

Qtd. de dados de cada classe (teste)
	Classe ham:  1352
	Classe spam:  196


## Tudo pronto

Dessa forma, com os dados do nosso dataset preparado, podemos agora ir para os outros notebooks para rodar os métodos de classificação.